# Imports

In [36]:
from qiskit.circuit.library.standard_gates import RXGate, RZGate, CXGate, CZGate, SGate, HGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from typing import Dict, Tuple, List
import matplotlib
from qiskit import assemble, Aer
from qiskit import *
from qiskit.visualization import plot_histogram
import math
import numpy as np
from numpy import linalg as LA
from qiskit.opflow import X, Z, I, H, Y
from qiskit.algorithms.optimizers import L_BFGS_B
from qiskit.quantum_info import Pauli

# Setting the BFGS optimazer and Backend

In [2]:
bfgs_optimizer = L_BFGS_B(maxiter=60)

simulator_backend = Aer.get_backend('aer_simulator') 

# Implement the ansatz

In [3]:
def anzats_circ1(thetas, D2, in_state):
    qr = QuantumRegister(4, name="q")
    qc = QuantumCircuit(qr)
    qc.initialize(in_state)
        
    for d in range(D2):
        qc.append(RXGate(thetas[0]), [qr[0]])
        qc.append(RXGate(thetas[1]), [qr[1]])
        qc.append(RXGate(thetas[2]), [qr[2]])
        qc.append(RXGate(thetas[3]), [qr[3]])
        
        qc.append(RZGate(thetas[4]), [qr[0]])
        qc.append(RZGate(thetas[5]), [qr[1]])
        qc.append(RZGate(thetas[6]), [qr[2]])
        qc.append(RZGate(thetas[7]), [qr[3]])
        
        qc.append(CZGate(), [qr[0], qr[1]])
        qc.append(CZGate(), [qr[1], qr[2]])
        qc.append(CZGate(), [qr[2], qr[3]])
        qc.barrier(qr)
    
    qc.append(RXGate(thetas[0]), [qr[0]])
    qc.append(RXGate(thetas[1]), [qr[1]])
    qc.append(RXGate(thetas[2]), [qr[2]])
    qc.append(RXGate(thetas[3]), [qr[3]])

    qc.append(RZGate(thetas[4]), [qr[0]])
    qc.append(RZGate(thetas[5]), [qr[1]])
    qc.append(RZGate(thetas[6]), [qr[2]])
    qc.append(RZGate(thetas[7]), [qr[3]])
       
    return qc

def anzats_circ1_uninitialized(thetas, D2):
    qr = QuantumRegister(4, name="q")
    qc = QuantumCircuit(qr)
        
    for d in range(D2):
        qc.append(RXGate(thetas[0]), [qr[0]])
        qc.append(RXGate(thetas[1]), [qr[1]])
        qc.append(RXGate(thetas[2]), [qr[2]])
        qc.append(RXGate(thetas[3]), [qr[3]])
        
        qc.append(RZGate(thetas[4]), [qr[0]])
        qc.append(RZGate(thetas[5]), [qr[1]])
        qc.append(RZGate(thetas[6]), [qr[2]])
        qc.append(RZGate(thetas[7]), [qr[3]])
        
        qc.append(CZGate(), [qr[0], qr[1]])
        qc.append(CZGate(), [qr[1], qr[2]])
        qc.append(CZGate(), [qr[2], qr[3]])
        qc.barrier(qr)
    
    qc.append(RXGate(thetas[0]), [qr[0]])
    qc.append(RXGate(thetas[1]), [qr[1]])
    qc.append(RXGate(thetas[2]), [qr[2]])
    qc.append(RXGate(thetas[3]), [qr[3]])

    qc.append(RZGate(thetas[4]), [qr[0]])
    qc.append(RZGate(thetas[5]), [qr[1]])
    qc.append(RZGate(thetas[6]), [qr[2]])
    qc.append(RZGate(thetas[7]), [qr[3]])
    
    
    return qc

def anzats_circ2(phis, D1, in_state):
    qr = QuantumRegister(4, name="q")
    cr = ClassicalRegister(4)
    qc = QuantumCircuit(qr, cr)
    qc.initialize(in_state)
    
    for d in range(D1):
        qc.append(RXGate(phis[0]), [qr[2]])
        qc.append(RXGate(phis[1]), [qr[3]])
        
        qc.append(RZGate(phis[2]), [qr[2]])
        qc.append(RZGate(phis[3]), [qr[3]])
        
        qc.append(CZGate(), [qr[2], qr[3]])
        qc.barrier(qr)
    return qc

# Choose k orthogonal states(computational basis)

In [4]:
def get_k_basis(k, n):
    full_basis = np.identity(n)
    return full_basis[:k]

# Generating the hamiltonians

In [43]:
H2_molecule_Hamiltonian =   -0.8105479805373279 * (I^I^I^I) \
                            + 0.1721839326191554 * (I^I^I^Z) \
                            - 0.22575349222402372 * (I^I^Z^I) \
                            + 0.17218393261915543 * (I^Z^I^I) \
                            - 0.2257534922240237 * (Z^I^I^I) \
                            + 0.12091263261776627 * (I^I^Z^Z) \
                            + 0.16892753870087907 * (I^Z^I^Z) \
                            + 0.045232799946057826 * (Y^Y^Y^Y) \
                            + 0.045232799946057826 * (X^X^Y^Y) \
                            + 0.045232799946057826 * (Y^Y^X^X) \
                            + 0.045232799946057826 * (X^X^X^X) \
                            + 0.1661454325638241 * (Z^I^I^Z) \
                            + 0.1661454325638241 * (I^Z^Z^I) \
                            + 0.17464343068300453 * (Z^I^Z^I) \
                            + 0.12091263261776627 * (Z^Z^I^I)

def create_pauli_string_with_pauli_op_on_index_i(pauli_op, i, qubits_num):
    if i == 1:
        pauli_string = pauli_op
        for qubit in range(qubits_num - 1):
            pauli_string = pauli_string ^ I
        return pauli_string
    
    pauli_string = I
    for qubit in range(2, qubits_num + 1):
        if qubit == i:
            pauli_string = pauli_string ^ pauli_op
        else:
            pauli_string = pauli_string ^ I
            
    return pauli_string

def create_pauli_string_with_pauli_ops_on_index_i_and_j(pauli_op_second, i, pauli_op_first, j, qubits_num):
    if j == 1:
        pauli_string = pauli_op_first
        for qubit in range(2, qubits_num + 1):
            if qubit == i:
                pauli_string = pauli_string ^ pauli_op_second
            else:
                pauli_string = pauli_string ^ I
        return pauli_string
    
    pauli_string = I
    for qubit in range(2, qubits_num + 1):
        if qubit == j:
            pauli_string = pauli_string ^ pauli_op_first
        elif qubit == i:
            pauli_string = pauli_string ^ pauli_op_second
        else:
            pauli_string = pauli_string ^ I
            
    return pauli_string
    
def get_Ising_model_hamiltonian():
    hamiltonian = I
    for qubit in range(QUBITS_NUM - 1):
        hamiltonian = hamiltonian^I
    hamiltonian = 0 * hamiltonian
    
    for i in range(1, QUBITS_NUM + 1):
        a_i = np.random.random_sample()
        x_i = create_pauli_string_with_pauli_op_on_index_i(X, i, QUBITS_NUM)
        hamiltonian = hamiltonian + a_i * x_i
        for j in range(1, i):
            J_ij = np.random.random_sample()
            z_ij = create_pauli_string_with_pauli_ops_on_index_i_and_j(Z, i, Z, j, QUBITS_NUM)
            hamiltonian = hamiltonian + J_ij * z_ij
    
    return hamiltonian
    
def get_hamiltonian(hamiltonian_type):
    if hamiltonian_type == "Ising Model":
        return get_Ising_model_hamiltonian()
    assert hamiltonian_type == "Molecular", "hamiltonian_type has to be Ising Model or Molecular"
    return H2_molecule_Hamiltonian

# Expectation Value 

### convert hamiltonian to pauli strings

In [6]:
reducing_to_pauli_z_dict = {
    Pauli('I'): Pauli('I'),
    Pauli('Z'): Pauli('Z'),
    Pauli('X'): Pauli('Z'),
    Pauli('Y'): Pauli('Z')
} 

In [7]:
def transfrom_hamiltonian_into_pauli_string(hamiltonian):
    pauli_operators = hamiltonian.to_pauli_op().settings['oplist']
    pauli_strings = list(map(lambda pauli_operator: pauli_operator.primitive, pauli_operators))
    pauli_coeffs = list(map(lambda pauli_operator: pauli_operator.coeff, pauli_operators))
    return (pauli_strings, pauli_coeffs)

def reduce_pauli_matrixes_into_sigma_z(pauli_string):
    for matrix_index in range(QUBITS_NUM):
        pauli_matrix = pauli_string[matrix_index]
        pauli_string[matrix_index].insert(reducing_to_pauli_z_dict[pauli_matrix])
    
    return pauli_string

def get_z_reduction_for_pauli_string(qc, pauli_string):
    qr = QuantumRegister(4, name="q")
    exdend_qc = QuantumCircuit(qr)
    pauli_string = str(pauli_string)
    for qubit_index, pauli_matrix in enumerate(pauli_string):
        if pauli_matrix == "X":
            exdend_qc.append(HGate(), [qr[qubit_index]])
        elif pauli_matrix == "Y":
            exdend_qc.append(HGate(), [qr[qubit_index]])
            exdend_qc.append(SGate(), [qr[qubit_index]])
    qc = qc.compose(exdend_qc)
    return qc

### probabilities distribution

In [8]:
def get_probability_distribution(counts: Dict) -> Dict:
    proba_distribution = {state: (count / NUM_SHOTS) for state, count in counts.items()}
    return proba_distribution

def calculate_probabilities_of_measurments_in_computational_basis(quantum_state_circuit) -> Dict:
    quantum_state_circuit.measure_all()
    
    transpiled_quantum_state_circuit = transpile(quantum_state_circuit, simulator_backend) 
    Qobj = assemble(transpiled_quantum_state_circuit)
    result = simulator_backend.run(Qobj).result()
    counts = result.get_counts(quantum_state_circuit)
    
    return get_probability_distribution(counts)

### Expectation value from probabilities

In [9]:
def sort_probas_dict_by_qubits_string_keys(proba_distribution: Dict) -> Dict:
    return dict(sorted(proba_distribution.items()))

def reset_power_of_minus_1(power_of_minus_1):
    power_of_minus_1 = 0
    return power_of_minus_1

def calculate_expectation_value_of_pauli_string_by_measurments_probas(pauli_string, probas_distribution):
    pauli_string_expectation_value = 0
    power_of_minus_1 = 0
    
    sorted_probas_distribuition = sort_probas_dict_by_qubits_string_keys(probas_distribution)
    for qubits_string, proba in sorted_probas_distribuition.items():
        for string_index in range(QUBITS_NUM):
            if(str(qubits_string[string_index])=="1" and str(pauli_string[string_index])=="Z"):
                power_of_minus_1 += 1
            
        pauli_string_expectation_value += pow(-1, power_of_minus_1)*proba
        power_of_minus_1 = reset_power_of_minus_1(power_of_minus_1)
        
    return pauli_string_expectation_value

def get_expectation_value(pauli_string, probas_distribution):
        return calculate_expectation_value_of_pauli_string_by_measurments_probas(
                                                                                pauli_string, probas_distribution)
    

# Calculating the first target function

In [10]:
def calc_target_func1(thetas, basis, D2, Ham):
    target_func = 0
    pauli_strings, pauli_coeffs = transfrom_hamiltonian_into_pauli_string(Ham)
    
    for j in basis:
        total_expectation_value = 0
        for pauli_index, pauli_string in enumerate(pauli_strings):
            qc = anzats_circ1(thetas, D2, j)
            qc = get_z_reduction_for_pauli_string(qc, pauli_string)
            probas_distribution = calculate_probabilities_of_measurments_in_computational_basis(qc)
            total_expectation_value += pauli_coeffs[pauli_index] * get_expectation_value(pauli_string, probas_distribution)
        
        target_func += total_expectation_value
        
    return target_func

def objective_func1(thetas):
    target_func = calc_target_func1(thetas, basis, D2, Ham)
    return target_func

# Calculating the second target function

In [11]:
def calc_target_func2(thetas_opt, phis, in_state, D1, D2, Ham):
    target_func = 0

    qc2 = anzats_circ2(phis, D1, in_state)
    qc1 = anzats_circ1_uninitialized(thetas_opt, D2)
   
    pauli_strings, pauli_coeffs = transfrom_hamiltonian_into_pauli_string(Ham)
    
    total_expectation_value = 0
    for pauli_index, pauli_string in enumerate(pauli_strings):
        qc = qc2.compose(qc1)
        qc = get_z_reduction_for_pauli_string(qc, pauli_string)
        probas_distribution = calculate_probabilities_of_measurments_in_computational_basis(qc)
        total_expectation_value += pauli_coeffs[pauli_index] * get_expectation_value(pauli_string, probas_distribution)
            
    target_func += total_expectation_value
        
    return target_func

def objective_func2(phis):
    in_state = basis[i]
    target_func2 = calc_target_func2(thetas_opt, phis, in_state, D1, D2, Ham)
    print("target func:")
    print(target_func2)
    return target_func2

def objective_func2_neg(phis):
    return -1*objective_func2(phis)

# Implementing SSVQE

In [12]:
def SSVQE(qubits_num, k, d1, d2, hamiltonian):  
    global QUBITS_NUM
    QUBITS_NUM = qubits_num
      
    global n
    n = 2 ** QUBITS_NUM
    
    global basis
    basis = get_k_basis(k,n)
    
    global D1, D2
    D1 = d1
    D2 = d2
    
    global Ham
    Ham = get_hamiltonian(hamiltonian)
    print("hamiltonian:")
    print(Ham)
    
    point, value, nfev = bfgs_optimizer.optimize(8,objective_func1,initial_point=np.zeros(8))
    print(point)
    print("---point---")
    print(value)
    print("---value---")

    global thetas_opt
    thetas_opt = point
    
    global i
    i = np.random.randint(0,k)
    
    point, value, nfev = bfgs_optimizer.optimize(4, objective_func2_neg, initial_point=np.array([1, 1, 1, 1]))
    print(point)
    print("---point---")
    print(value)
    print("---value---")
    
    base_energy = -value
    print("The SSVQE result for the minimal energy of the Hamiltonian given is:")
    print(base_energy)

# SSVQE With Ising Model Hamiltonian

In [13]:
#Backend number of shots
NUM_SHOTS = 1024

qubits_num = 4
k = 2
d1 = 1
d2 = 1
hamiltonian = "Ising Model"
SSVQE(qubits_num, k, d1, d2, hamiltonian)

hamiltonian:
0.0 * IIII
+ 0.38010445226830414 * XIII
+ 0.00669325360698847 * IXII
+ 0.6632600826916025 * ZZII
+ 0.40032425353201817 * IIXI
+ 0.20462428656706544 * ZIZI
+ 0.04515814137505447 * IZZI
+ 0.24340542969934842 * IIIX
+ 0.1948306591425536 * ZIIZ
+ 0.12387423319840851 * IZIZ
+ 0.30329267218152145 * IIZZ
[0. 0. 0. 0. 0. 0. 0. 0.]
---point---
3.005704871723287
---value---
target func:
0.34318572799405145
target func:
0.32956363781332176
target func:
0.31933350002304406
target func:
0.3192731707108183
target func:
0.3358180426404973
target func:
0.4115690053612807
target func:
0.40076483010793995
target func:
0.3902680677162079
target func:
0.39442187670063467
target func:
0.393987414319463
target func:
0.3519692906711797
target func:
0.3432169361032863
target func:
0.3497980685064266
target func:
0.34088078709974906
target func:
0.34536730312879876
target func:
0.3392919708577082
target func:
0.34482790324840884
target func:
0.3219039028767133
target func:
0.3290882626881006
targe

# SSVQE With H2 Molecule Hamiltonian

In [47]:
#Backend number of shots
NUM_SHOTS = 1024

qubits_num = 4
k = 2
d1 = 1
d2 = 1
hamiltonian = "Molecular"
SSVQE(qubits_num, k, d1, d2, hamiltonian)

hamiltonian:
-0.8105479805373279 * IIII
+ 0.1721839326191554 * IIIZ
- 0.22575349222402372 * IIZI
+ 0.17218393261915543 * IZII
- 0.2257534922240237 * ZIII
+ 0.12091263261776627 * IIZZ
+ 0.16892753870087907 * IZIZ
+ 0.045232799946057826 * YYYY
+ 0.045232799946057826 * XXYY
+ 0.045232799946057826 * YYXX
+ 0.045232799946057826 * XXXX
+ 0.1661454325638241 * ZIIZ
+ 0.1661454325638241 * IZZI
+ 0.17464343068300453 * ZIZI
+ 0.12091263261776627 * ZZII
[0. 0. 0. 0. 0. 0. 0. 0.]
---point---
-0.11003360771268023
---value---
target func:
-0.19882026624932533
target func:
-0.21267128236603686
target func:
-0.22453680725445338
target func:
-0.18513278236775396
target func:
-0.18738258761654528
target func:
0.12409848487331976
target func:
0.12591527136563593
target func:
0.12396931365974379
target func:
0.12890222544812963
target func:
0.11638593887832321
target func:
-0.053085677489874444
target func:
-0.05292007594123002
target func:
-0.04051104550806453
target func:
-0.05884838137675752
target func